In [43]:
import pandas as pd
import numpy as np
file_path = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData'

In [2]:
boston = pd.read_csv(file_path + '/clean/boston_weekly_data_withblack.csv', low_memory = False)
seattle = pd.read_csv(file_path + '/clean/seattle_weekly_data_withblack.csv', low_memory = False)
nyc = pd.read_csv(file_path + '/clean/nyc_weekly_data_withblack.csv', low_memory = False)
la = pd.read_csv(file_path + '/clean/la_weekly_data_withblack.csv', low_memory = False)

In [4]:
la = la.rename(columns={"new_case": "case_count"})
boston = boston.rename(columns={"weekly_cases": "case_count"})

In [5]:
nyc = nyc.drop(['CASERATE', 'PCTPOS', 'y_hat'], axis = 1)
la = la.drop(['caserate', 'y_hat'], axis=1)

In [7]:
nyc[nyc.isna().any(axis=1)]

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,week_no,case_count,percent_black


In [8]:
la[la.isna().any(axis=1)]

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,week_no,percent_black


In [9]:
boston['week'] = pd.to_datetime(boston['week'])
seattle['week'] = pd.to_datetime(seattle['week'])
nyc['week'] = pd.to_datetime(nyc['week'])
la['week'] = pd.to_datetime(la['week'])

# Pre-treatment Data Preparation (2018/01/01 - 2019/12/30)

In [10]:
boston_1st = boston[(boston['week'] >= '2018-01-01') & (boston['week'] <= '2019-12-30')]
seattle_1st = seattle[(seattle['week'] >= '2018-01-01') & (seattle['week'] <= '2019-12-30')]
nyc_1st = nyc[(nyc['week'] >= '2018-01-01') & (nyc['week'] <= '2019-12-30')]
la_1st = la[(la['week'] >= '2018-01-01') & (la['week'] <= '2019-12-30')]

In [11]:
nyc_1st['city'] = 'nyc'
la_1st['city'] = 'la'
boston_1st['city'] = 'boston'
seattle_1st['city'] = 'seattle'

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/765401552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_1st['city'] = 'nyc'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/765401552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_1st['city'] = 'la'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/765401552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [12]:
dflist = [boston_1st, seattle_1st, nyc_1st, la_1st]

for df in dflist:
    # Extract year and month, then combine them into a single column
    df['year_month'] = df['week'].dt.to_period('M')


/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/3730094990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['week'].dt.to_period('M')
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/3730094990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['week'].dt.to_period('M')
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/3730094990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [16]:
year_month_dummies = pd.get_dummies(boston_1st['year_month'], prefix='ym')
boston_1st = pd.concat([boston_1st, year_month_dummies], axis=1)
boston_1st.head()

,Unnamed: 0,ZCTA5CE10,week,total_count,total_population,crime_rate,year,e/pop,un_rate,p_rate,...,ym_2019-03,ym_2019-04,ym_2019-05,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12
0,0,2108,2018-01-07,12,4218,28.449502,2018,61.2,8.3,10.8,...,0,0,0,0,0,0,0,0,0,0
1,1,2108,2018-01-14,16,4218,37.932670,2018,61.2,8.3,10.8,...,0,0,0,0,0,0,0,0,0,0
2,2,2108,2018-01-21,18,4218,42.674253,2018,61.2,8.3,10.8,...,0,0,0,0,0,0,0,0,0,0
3,3,2108,2018-01-28,16,4218,37.932670,2018,61.2,8.3,10.8,...,0,0,0,0,0,0,0,0,0,0
4,4,2108,2018-02-04,15,4218,35.561878,2018,61.2,8.3,10.8,...,0,0,0,0,0,0,0,0,0,0


In [21]:
year_month_dummies = pd.get_dummies(seattle_1st['year_month'], prefix='ym')
seattle_1st = pd.concat([seattle_1st, year_month_dummies], axis=1)
seattle_1st.head()

,Unnamed: 0,ZCTA5CE10,week,total_count,total_population,crime_rate,year,e/pop,un_rate,p_rate,...,ym_2019-03,ym_2019-04,ym_2019-05,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12
0,0,98101,2018-01-07,126,16302,77.291130,2018,67.5,3.8,14.7,...,0,0,0,0,0,0,0,0,0,0
1,1,98101,2018-01-14,132,16302,80.971660,2018,67.5,3.8,14.7,...,0,0,0,0,0,0,0,0,0,0
2,2,98101,2018-01-21,127,16302,77.904552,2018,67.5,3.8,14.7,...,0,0,0,0,0,0,0,0,0,0
3,3,98101,2018-01-28,110,16302,67.476383,2018,67.5,3.8,14.7,...,0,0,0,0,0,0,0,0,0,0
4,4,98101,2018-02-04,141,16302,86.492455,2018,67.5,3.8,14.7,...,0,0,0,0,0,0,0,0,0,0


In [22]:
year_month_dummies = pd.get_dummies(nyc_1st['year_month'], prefix='ym')
nyc_1st = pd.concat([nyc_1st, year_month_dummies], axis=1)
nyc_1st.head()

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,...,ym_2019-03,ym_2019-04,ym_2019-05,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12
0,0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
2,2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
3,3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
year_month_dummies = pd.get_dummies(la_1st['year_month'], prefix='ym')
la_1st = pd.concat([la_1st, year_month_dummies], axis=1)
la_1st.head()

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,...,ym_2019-03,ym_2019-04,ym_2019-05,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12
0,0,2018-01-07,90001,21.0,57652.0,3.642545,56.1,9.7,28.8,38521,...,0,0,0,0,0,0,0,0,0,0
1,1,2018-01-14,90001,14.0,57652.0,2.428363,56.1,9.7,28.8,38521,...,0,0,0,0,0,0,0,0,0,0
2,2,2018-02-04,90001,23.0,57652.0,3.989454,56.1,9.7,28.8,38521,...,0,0,0,0,0,0,0,0,0,0
3,3,2018-02-11,90001,15.0,57652.0,2.601818,56.1,9.7,28.8,38521,...,0,0,0,0,0,0,0,0,0,0
4,4,2018-02-18,90001,21.0,57652.0,3.642545,56.1,9.7,28.8,38521,...,0,0,0,0,0,0,0,0,0,0


In [24]:
nycla_1st = pd.concat([nyc_1st, la_1st], axis=0)
nycboston_1st = pd.concat([nyc_1st, boston_1st], axis=0)
nycseattle_1st = pd.concat([nyc_1st, seattle_1st], axis=0)

In [25]:
nycla_1st.head()

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,...,ym_2019-03,ym_2019-04,ym_2019-05,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12
0,0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
2,2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
3,3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
nycseattle_1st.tail()

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,...,ym_2019-06,ym_2019-07,ym_2019-08,ym_2019-09,ym_2019-10,ym_2019-11,ym_2019-12,year,e/pop,y_hat
7456,7456,2019-12-01,98199,19.0,22359.0,8.497697,NaN,3.4,5.5,108328.0,...,0,0,0,0,0,0,1,2019.0,69.0,10.280910
7457,7457,2019-12-08,98199,19.0,22359.0,8.497697,NaN,3.4,5.5,108328.0,...,0,0,0,0,0,0,1,2019.0,69.0,10.271513
7458,7458,2019-12-15,98199,8.0,22359.0,3.577978,NaN,3.4,5.5,108328.0,...,0,0,0,0,0,0,1,2019.0,69.0,10.262115
7459,7459,2019-12-22,98199,21.0,22359.0,9.392191,NaN,3.4,5.5,108328.0,...,0,0,0,0,0,0,1,2019.0,69.0,10.252717
7460,7460,2019-12-29,98199,10.0,22359.0,4.472472,NaN,3.4,5.5,108328.0,...,0,0,0,0,0,0,1,2019.0,69.0,10.243319


In [26]:
nycla_1st.to_csv(file_path + '/clean/linear_regression/nyc_la_parallel.csv', index=False)
nycboston_1st.to_csv(file_path + '/clean/linear_regression/nyc_boston_parallel.csv', index=False)
nycseattle_1st.to_csv(file_path + '/clean/linear_regression/nyc_seattle_parallel.csv', index=False)

## NYC - Seattle Full Data Preparation

In [49]:
nyc_prepost = nyc[(nyc['week'] >= '2018-01-01') & (nyc['week'] < '2020-01-01') | (nyc['week'] >= '2022-05-01') & (nyc['week'] <= '2023-05-30')] 

seattle_prepost = seattle[(seattle['week'] >= '2018-01-01') & (seattle['week'] < '2020-01-01') | (seattle['week'] >= '2022-05-01') & (seattle['week'] <= '2023-05-30')]

nyc_prepost['treated'] = 1
seattle_prepost['treated'] = 0

nyc_prepost['post'] = np.where(nyc_prepost['week'] >= '2020-01-01', 1, 0)
seattle_prepost['post'] = np.where(seattle_prepost['week'] >= '2020-01-01', 1, 0)

nyc_seattle_prepost = pd.concat([nyc_prepost, seattle_prepost], axis=0)
nyc_seattle_prepost.head()

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/1841592527.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_prepost['treated'] = 1
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/1841592527.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seattle_prepost['treated'] = 0
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_15701/1841592527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,Unnamed: 0,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,week_no,case_count,percent_black,treated,post,year,e/pop,y_hat
0,0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,1,0.0,0.090458,1,0,NaN,NaN,NaN
1,1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,2,0.0,0.090458,1,0,NaN,NaN,NaN
2,2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,5,0.0,0.090458,1,0,NaN,NaN,NaN
3,3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,6,0.0,0.090458,1,0,NaN,NaN,NaN
4,4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,151628.0,7,0.0,0.090458,1,0,NaN,NaN,NaN


In [50]:
nyc_seattle_prepost.to_csv(file_path + '/clean/nyc_seattle_stata.csv', index = False)

## Save PrePost Data for All Cities

In [51]:
boston_prepost = boston[(boston['week'] >= '2018-01-01') & (boston['week'] < '2020-01-01') | (boston['week'] >= '2022-05-01') & (boston['week'] <= '2023-05-30')] 
seattle_prepost = seattle[(seattle['week'] >= '2018-01-01') & (seattle['week'] < '2020-01-01') | (seattle['week'] >= '2022-05-01') & (seattle['week'] <= '2023-05-30')]
nyc_prepost = nyc[(nyc['week'] >= '2018-01-01') & (nyc['week'] < '2020-01-01') | (nyc['week'] >= '2022-05-01') & (nyc['week'] <= '2023-05-30')]
la_prepost = la[(la['week'] >= '2018-01-01') & (la['week'] < '2020-01-01') | (la['week'] >= '2022-05-01') & (la['week'] <= '2023-05-30')]


In [54]:
boston_prepost.describe()

,Unnamed: 0,ZCTA5CE10,total_count,total_population,crime_rate,year,e/pop,un_rate,p_rate,in_median,in_mean,week_no,case_count,percent_black
count,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000,5021.0,5021.000000
mean,4392.664210,2143.829118,50.055567,24001.184425,27.563921,2019.899422,62.748835,6.802828,19.160088,78118.438956,78118.438956,125.144991,0.0,0.143535
std,2559.297899,64.793637,39.517033,15501.226464,23.900919,1.927067,10.903533,3.489885,9.845709,35127.139178,35127.139178,99.992156,0.0,0.176955
min,0.000000,2108.000000,0.000000,1221.000000,0.000000,2018.000000,22.200000,0.800000,2.300000,30551.000000,30551.000000,1.000000,0.0,0.008841
25%,2223.000000,2116.000000,18.000000,8417.000000,13.968590,2018.000000,55.800000,4.500000,11.600000,52474.000000,52474.000000,41.000000,0.0,0.039671
50%,4325.000000,2126.000000,41.000000,24705.000000,20.687710,2019.000000,64.400000,6.000000,18.000000,65250.000000,65250.000000,82.000000,0.0,0.052152
75%,6548.000000,2135.000000,75.000000,31903.000000,35.620320,2022.000000,69.100000,9.200000,24.800000,100915.000000,100915.000000,243.000000,0.0,0.200956
max,8874.000000,2467.000000,220.000000,60592.000000,196.560197,2023.000000,82.300000,14.600000,40.200000,163068.000000,163068.000000,282.000000,0.0,0.706167


In [55]:
nyc_prepost.describe()

,Unnamed: 0,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,week_no,case_count,percent_black
count,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000,29120.000000
mean,25494.513496,10810.361401,51.177060,47746.539870,11.103797,59.905515,6.548620,16.382129,74680.173661,105340.292891,123.886779,30.732563,0.182870
std,14728.908416,572.772504,38.068352,26809.365427,8.460942,8.422736,2.933021,9.718924,37254.663447,56903.676745,99.795987,62.381315,0.214342
min,0.000000,10001.000000,0.000000,1523.000000,0.000000,18.000000,0.000000,0.000000,21149.000000,35479.000000,1.000000,0.000000,0.001343
25%,12724.750000,10301.000000,21.000000,27632.000000,6.139992,54.800000,4.500000,9.200000,51183.000000,70551.000000,41.000000,0.000000,0.029426
50%,25449.500000,11105.000000,43.000000,43045.000000,9.401007,59.500000,6.100000,13.400000,66570.000000,88365.000000,81.000000,0.000000,0.077516
75%,38295.250000,11360.000000,75.000000,68182.000000,13.634814,64.900000,7.800000,22.400000,88803.000000,112310.000000,242.000000,25.959761,0.263631
max,51020.000000,11697.000000,225.000000,112750.000000,118.951380,88.100000,15.800000,45.900000,250000.000000,367343.000000,282.000000,502.679324,0.838705


In [56]:
la_prepost.describe()

,Unnamed: 0,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,week_no,percent_black
count,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000,21269.000000
mean,18735.877568,90548.259627,32.856364,37224.181391,10.025817,61.484804,6.979096,16.955282,69882.410457,100944.420189,5108.064554,122.565612,0.089975
std,10889.160384,598.908046,27.839067,19942.994373,14.731972,6.436538,2.109029,9.707734,32684.461845,53767.631872,9813.849946,99.000637,0.125029
min,0.000000,90001.000000,0.000000,2653.000000,0.000000,38.600000,2.300000,3.500000,15149.000000,36307.000000,0.000000,1.000000,0.006172
25%,9310.000000,90037.000000,8.000000,24481.000000,3.687452,58.000000,5.500000,9.800000,46397.000000,63193.000000,0.000000,41.000000,0.027309
50%,18620.000000,90247.000000,30.000000,34649.000000,8.247515,61.300000,6.700000,15.200000,62791.000000,84763.000000,0.000000,80.000000,0.046526
75%,28323.000000,91307.000000,50.000000,48170.000000,11.837071,65.500000,8.300000,20.800000,86031.000000,121050.000000,6165.000000,241.000000,0.082422
max,37583.000000,91803.000000,189.000000,106042.000000,266.091334,76.200000,13.300000,51.500000,210833.000000,321822.000000,43395.000000,282.000000,0.755775


In [57]:
seattle_prepost.describe()

,Unnamed: 0,ZCTA5CE10,total_count,total_population,crime_rate,year,e/pop,un_rate,p_rate,in_median,in_mean,case_count,week_no,y_hat,percent_black
count,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000,4347.000000
mean,3801.690361,98126.074074,47.155510,31486.777778,16.521243,2019.875776,69.181481,4.203704,11.392593,85583.777778,85583.777778,16.270531,123.838509,16.612379,0.072897
std,2206.547546,25.117332,30.805615,11614.937979,14.784333,1.923462,6.621830,1.384056,6.213421,20061.133518,20061.133518,37.404285,99.828950,7.316637,0.062233
min,0.000000,98101.000000,0.000000,15052.000000,0.000000,2018.000000,53.500000,2.000000,4.000000,43601.000000,43601.000000,0.000000,1.000000,3.681236,0.013087
25%,1939.500000,98107.000000,24.000000,22359.000000,8.610792,2018.000000,65.100000,3.200000,6.100000,67556.000000,67556.000000,0.000000,41.000000,12.408474,0.023315
50%,3759.000000,98118.000000,45.000000,28088.000000,13.915948,2019.000000,69.500000,3.800000,9.200000,86525.000000,86525.000000,0.000000,81.000000,15.043061,0.054084
75%,5699.500000,98136.000000,69.000000,41268.000000,19.635072,2022.000000,72.600000,5.000000,14.700000,107276.000000,107276.000000,16.000000,242.000000,20.004740,0.115840
max,7638.000000,98199.000000,147.000000,54457.000000,90.172985,2023.000000,81.700000,7.300000,30.500000,115265.000000,115265.000000,312.000000,282.000000,35.945197,0.247646


In [52]:
nyc_prepost.to_csv(file_path + '/clean/nyc_prepost.csv')
la_prepost.to_csv(file_path + '/clean/la_prepost.csv')
boston_prepost.to_csv(file_path + '/clean/boston_prepost.csv')
seattle_prepost.to_csv(file_path + '/clean/seattle_prepost.csv')
